# Importing

In [1]:
import pyiast
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
import time
import os
import scipy.optimize as optim
import matplotlib.pyplot as plt

In [2]:
df_NAME=pd.read_csv("HEAT_0215.csv")

bins_CO2 = []
bins_H2S = []
for nam in df_NAME["NAME"]:
    f_tmp = open("iso_"+nam+"_saved.bin",'rb')
    bin_tmp = np.array(pickle.load(f_tmp))
    bins_CO2.append(bin_tmp[[False, True, True]]) ## Pick 1 = CO2, 2 = CH4 
    f_tmp.close()
for nam in df_NAME["NAME"]:
    f_tmp = open("iso_"+nam+"_saved.bin",'rb') ## 0 = H2S , 1 = CO2, 2 = CH4
    bin_tmp = np.array(pickle.load(f_tmp))
    bins_H2S.append(bin_tmp[[True, False, True]]) ## Pick 0 = H2S and 2 = CH4
    f_tmp.close()

In [5]:
Names_CO2=df_NAME["NAME"].to_numpy()
Names_H2S=df_NAME["NAME"].to_numpy()
dH_CO2 = np.array([df_NAME["CO2_Heat"], df_NAME["CH4_Heat"]]).T
dH_H2S = np.array([df_NAME["H2S_Heat"],df_NAME["CH4_Heat"]]).T

In [74]:
#P_targ = np.loadtxt('p_range.csv')/1E5
P_targ = np.linspace(0,50,251)

# iso_mix: IAST Calculation

In [75]:
Arrh = lambda T,dH ,T_ref: np.exp(-dH/8.3145*(1/T - 1/T_ref)) # Arrhenius equation (Clasius-Clapeyron Equation)

## Isothermal mixture isotherm
def iso_mix(P_par, T, iso_list, dH_list,Tref_list):
    P_norm = []
    for (p,dh,tref) in zip(P_par, dH_list,Tref_list):
        p_n = Arrh(T,dh,tref)*p 
        P_norm.append(p_n)
    P_norm_arr = np.array(P_norm)
    #print(P_norm_mat.T)
    if P_norm_arr.ndim > 1:
        for i in range(len(P_norm[0])):
            p_tmp = P_norm_arr[i,:]
            p_tmp[p_tmp<0.000001] = 0.000001
            q_IAST_tmp = pyiast.iast(p_tmp,
                                     iso_list,
                                     warningoff=True)
    else:
        try:
            p_tmp = P_norm_arr
            p_tmp[p_tmp<0.000001] = 0.000001
            #print(p_tmp)
            q_IAST_tmp = pyiast.iast(p_tmp,
                                    iso_list,
                                     warningoff=True)
        except:    
            try:
                #print('Initial guess error with P = ',P_par)
                x_IG = np.ones(len(p_tmp))/len(p_tmp)
                q_IAST_tmp = pyiast.iast(p_tmp,
                                        iso_list,adsorbed_mole_fraction_guess = x_IG,
                                        warningoff=True)
            except:
                try:
                    arg_min = np.argmin(p_tmp)
                    p_tmp[p_tmp<0.000001] = 0.000001
                    x_IG = 0.05*np.ones(len(p_tmp))
                    x_IG[arg_min] = 1 - 0.05*(len(p_tmp)-1)
                    #print(x_IG)
                    q_IAST_tmp = pyiast.iast(p_tmp,
                                            iso_list,adsorbed_mole_fraction_guess = x_IG,
                                            warningoff=True)

                except:
                    try:
                        arg_max = np.argmax(p_tmp)
                        p_tmp[p_tmp<0.000001] = 0.000001
                        x_IG = 0.05*np.ones(len(p_tmp))
                        x_IG[arg_max] = 1 - 0.05*(len(p_tmp)-1)
                        #print(x_IG)
                        q_IAST_tmp = pyiast.iast(p_tmp,
                                                iso_list,adsorbed_mole_fraction_guess = x_IG,
                                                warningoff=True)        
                    except:
                        try:
                            arg_max = np.argmax(p_tmp)
                            p_tmp[p_tmp<0.000001] = 0.000001
                            x_IG = 0.15*np.ones(len(p_tmp))
                            x_IG[arg_max] = 1 - 0.15*(len(p_tmp)-1)
                            #print(x_IG)
                            q_IAST_tmp = pyiast.iast(p_tmp,
                                                iso_list,adsorbed_mole_fraction_guess = x_IG,
                                                warningoff=True)
                        except:
                            try:
                                arg_min = np.argmin(p_tmp)
                                p_tmp[p_tmp<0.000001] = 0.000001
                                x_IG = 0.01*np.ones(len(p_tmp))
                                x_IG[arg_min] = 1 - 0.01*(len(p_tmp)-1)
                                #print(x_IG)
                                q_IAST_tmp = pyiast.iast(p_tmp,
                                            iso_list,adsorbed_mole_fraction_guess = x_IG,
                                            warningoff=True)

                            except:
                                try:
                                    arg_max = np.argmax(p_tmp)
                                    p_tmp[p_tmp<0.000001] = 0.000001
                                    x_IG = 0.01*np.ones(len(p_tmp))
                                    x_IG[arg_max] = 1 - 0.01*(len(p_tmp)-1)
                                    #print(x_IG)
                                    q_IAST_tmp = pyiast.iast(p_tmp,
                                                    iso_list,adsorbed_mole_fraction_guess = x_IG,
                                                warningoff=True)        
                                except:
                                    p_tmp[p_tmp<0.000001] = 0.000001
                                    x_IG = [0.9999, 0.0001]
                                    #print(x_IG)
                                    q_IAST_tmp = pyiast.iast(p_tmp,
                                                    iso_list,adsorbed_mole_fraction_guess = x_IG,
                                                warningoff=True)    
           
    return q_IAST_tmp

# Picking up the target adsorbents

In [76]:
AD_targ = [1,24, 25,57,113,153,156,250,340]

In [77]:
bins_targ_CO2 = []
bins_targ_H2S = []

dH_targ_CO2 = []
dH_targ_H2S = []

Names_targ  = []
for iii in AD_targ:
    print(Names_CO2[iii] , ' == ', Names_H2S[iii])
    bins_targ_CO2.append(bins_CO2[iii])
    bins_targ_H2S.append(bins_H2S[iii])
    
    dH_targ_CO2.append(dH_CO2[iii])
    dH_targ_H2S.append(dH_H2S[iii])
    
    Names_targ.append(Names_CO2[iii])
    

AEI_0  ==  AEI_0
APC_1  ==  APC_1
APC_2  ==  APC_2
CDO_1  ==  CDO_1
GIS_1  ==  GIS_1
ITV_0  ==  ITV_0
IWR_0  ==  IWR_0
PWN  ==  PWN
WEI_0  ==  WEI_0


# CO2 case

In [78]:
yFeed = np.array([0.5,0.5])
T_targ = 343
qCO2_dict = {'Pressure': P_targ}
qCH4_dict = {'Pressure': P_targ}
for isoo, dHH, Namm in zip(bins_targ_CO2, dH_targ_CO2, Names_targ):
    qCO2_list = []
    qCH4_list = []
    for pp in P_targ:
        qCO2,qCH4 = iso_mix(pp*yFeed,T_targ, isoo,dHH,[298,298])
        qCO2_list.append(qCO2)
        qCH4_list.append(qCH4)
    qCO2_dict[Namm] = qCO2_list
    qCH4_dict[Namm] = qCH4_list

In [79]:
#qCO2_dict
#qCH4_dict

In [80]:
df_qCO2 = pd.DataFrame(qCO2_dict)
df_qCH4 = pd.DataFrame(qCH4_dict)

df_qCO2.to_csv('IAST_targ_CASE1_CO2.csv',index =False)
df_qCH4.to_csv('IAST_targ_CASE1_CH4.csv',index =False)

In [81]:
df_qCO2

,Pressure,AEI_0,APC_1,APC_2,CDO_1,GIS_1,ITV_0,IWR_0,PWN,WEI_0
0,0.0,4.900540e-07,4.387536e-08,1.869126e-08,4.378803e-07,5.356312e-08,1.765975e-07,2.808513e-07,3.227109e-07,2.401796e-08
1,0.2,4.820812e-02,4.384455e-03,1.865300e-03,4.264178e-02,5.348777e-03,1.759716e-02,2.701443e-02,3.129893e-02,2.399085e-03
2,0.4,9.478898e-02,8.776696e-03,3.728134e-03,8.316371e-02,1.069210e-02,3.509059e-02,5.208730e-02,6.082430e-02,4.797435e-03
3,0.6,1.396749e-01,1.317642e-02,5.588257e-03,1.216643e-01,1.602882e-02,5.247990e-02,7.541967e-02,8.872385e-02,7.194923e-03
4,0.8,1.828754e-01,1.758334e-02,7.445430e-03,1.582467e-01,2.135784e-02,6.976474e-02,9.718542e-02,1.151295e-01,9.591426e-03
...,...,...,...,...,...,...,...,...,...,...
246,49.2,1.843625e+00,9.701310e-01,2.784369e-01,1.193202e+00,7.094724e-01,2.144418e+00,5.177536e-01,9.857899e-01,4.561373e-01
247,49.4,1.844525e+00,9.731098e-01,2.789700e-01,1.193519e+00,7.106158e-01,2.148146e+00,5.174433e-01,9.862727e-01,4.573772e-01
248,49.6,1.845416e+00,9.760802e-01,2.795002e-01,1.193833e+00,7.117524e-01,2.151853e+00,5.171324e-01,9.867512e-01,4.586130e-01
249,49.8,1.846298e+00,9.790424e-01,2.800275e-01,1.194143e+00,7.128822e-01,2.155541e+00,5.168209e-01,9.872252e-01,4.598445e-01


# H2S case

In [70]:
yFeed = np.array([0.5,0.5])
T_targ = 343
qH2S_dict = {'Pressure': P_targ}
qCH4_dict2 = {'Pressure': P_targ}
for isoo, dHH, Namm in zip(bins_targ_H2S, dH_targ_H2S, Names_targ):
    qH2S_list = []
    qCH4_list = []
    for pp in P_targ:
        qH2S,qCH4 = iso_mix(pp*yFeed,T_targ, isoo,dHH,[298,298])
        qH2S_list.append(qH2S)
        qCH4_list.append(qCH4)
    qH2S_dict[Namm] = qH2S_list
    qCH4_dict2[Namm] = qCH4_list

In [71]:
#qH2S_dict
#qCH4_dict2

In [72]:
df_qH2S = pd.DataFrame(qH2S_dict)
df_qCH4_2 = pd.DataFrame(qCH4_dict2)

df_qH2S.to_csv('IAST_targ_CASE2_H2S.csv',index =False)
df_qCH4_2.to_csv('IAST_targ_CASE2_CH4.csv',index =False)

In [73]:
df_qH2S

,Pressure,AEI_0,APC_1,APC_2,CDO_1,GIS_1,ITV_0,IWR_0,PWN,WEI_0
0,0.0,1.483096e-07,3.191731e-08,1.811023e-08,3.522393e-07,5.402572e-08,6.608670e-08,1.002552e-07,1.564986e-07,1.402566e-08
1,0.5,4.101403e-02,8.819171e-03,4.953709e-03,8.879597e-02,1.548209e-02,1.896941e-02,2.976172e-02,4.297727e-02,3.856119e-03
2,1.0,8.908761e-02,1.911616e-02,1.068371e-02,1.752560e-01,3.405284e-02,4.180043e-02,6.518829e-02,9.092270e-02,8.339292e-03
3,1.5,1.429225e-01,3.062774e-02,1.707996e-02,2.560315e-01,5.479049e-02,6.743403e-02,1.031829e-01,1.411236e-01,1.335357e-02
4,2.0,2.012639e-01,4.314653e-02,2.404838e-02,3.298940e-01,7.705258e-02,9.515772e-02,1.420986e-01,1.919611e-01,1.882064e-02
...,...,...,...,...,...,...,...,...,...,...
96,48.0,1.976615e+00,7.410952e-01,4.130358e-01,1.139163e+00,7.170627e-01,1.257767e+00,1.025408e+00,1.372498e+00,3.635612e-01
97,48.5,1.977811e+00,7.425645e-01,4.137775e-01,1.139565e+00,7.166470e-01,1.258981e+00,1.026528e+00,1.374052e+00,3.644701e-01
98,49.0,1.978952e+00,7.439878e-01,4.144955e-01,1.139949e+00,7.162046e-01,1.260126e+00,1.027615e+00,1.375558e+00,3.653557e-01
99,49.5,1.980038e+00,7.453664e-01,4.151903e-01,1.140316e+00,7.157366e-01,1.261204e+00,1.028672e+00,1.377018e+00,3.662185e-01
